In [1]:
#preparing data and mixing data
# hindi alpaca
# samvaad sarvam ai 
# gujarati all
# aya
# goal is to mix and prepare dataset for gujarati and hindi combined
# ideal data format is input and output


In [56]:
import pandas as pd
import json
import ast
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [15]:
output_location = "prepared_data_df/"
if not os.path.exists(output_location):
    os.makedirs(output_location)

In [3]:
from datasets import load_from_disk, load_dataset
dataset_name = "jayshah5696/alpaca-hindi"
dataset = load_dataset(dataset_name)

/Users/jshah/anaconda3/envs/hf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 627.70it/s]
Generating train split: 1 examples [00:00, 231.54 examples/s]


In [7]:
dataset['train']

Dataset({
    features: ['_data_files', '_fingerprint', '_format_columns', '_format_kwargs', '_format_type', '_output_all_columns', '_split'],
    num_rows: 1
})

In [9]:
df = pd.read_csv('/Users/jshah/Documents/GitHub/notebooks/Transformers/data/alpaca-hindi/df_modified.csv')


In [11]:
df.head()

,instruction,input,output,text,hindi_instruction,hindi_input,hindi_output,hindi_text
0,Give three tips for staying healthy.,NaN,1.Eat a balanced diet and make sure to include...,Below is an instruction that describes a task....,स्वस्थ रहने के लिए तीन सुझाव दें।.,NaN,1. एक संतुलित आहार खाएं और बहुत सारे फलों और स...,नीचे एक निर्देश है जो एक कार्य का वर्णन करता ह...
1,What are the three primary colors?,NaN,"The three primary colors are red, blue, and ye...",Below is an instruction that describes a task....,तीन प्राथमिक रंग क्या हैं?,NaN,"तीन प्राथमिक रंग लाल, नीले और पीले हैं।.",नीचे एक निर्देश है जो एक कार्य का वर्णन करता ह...
2,Describe the structure of an atom.,NaN,"An atom is made up of a nucleus, which contain...",Below is an instruction that describes a task....,एक परमाणु की संरचना का वर्णन करें।.,NaN,"एक परमाणु एक नाभिक से बना है, जिसमें प्रोटॉन औ...",नीचे एक निर्देश है जो एक कार्य का वर्णन करता ह...
3,How can we reduce air pollution?,NaN,There are a number of ways to reduce air pollu...,Below is an instruction that describes a task....,हम वायु प्रदूषण को कैसे कम कर सकते हैं?,NaN,"वायु प्रदूषण को कम करने के कई तरीके हैं, जैसे ...",नीचे एक निर्देश है जो एक कार्य का वर्णन करता ह...
4,Describe a time when you had to make a difficu...,NaN,I had to make a difficult decision when I was ...,Below is an instruction that describes a task....,जब आप एक कठिन निर्णय करना चाहते थे तब एक समय क...,NaN,जब मैं एक निर्माण कंपनी में एक परियोजना प्रबंध...,नीचे एक निर्देश है जो एक कार्य का वर्णन करता ह...


In [35]:
hi_columns = [col for col in df.columns if "hindi" in col]
df_hindi = df[hi_columns]
en_columns = [col for col in df.columns if "hindi" not in col]
df_en = df[en_columns]
df_hindi['input'] = df_hindi.apply(lambda x: x['hindi_instruction'] if pd.isna(x['hindi_input']) else x['hindi_instruction']+'\n'+x['hindi_input'],axis=1)
df_hindi['output']=df_hindi['hindi_output']
df_hindi['language']='hindi'
df_en['input'] = df_en.apply(lambda x: x['instruction'] if pd.isna(x['input']) else x['instruction']+'\n'+x['input'],axis=1)
df_en['language']='english'
req_cols = ['input','output',"language"]
df_alpaca_hindi_mix_input_output_format = pd.concat([df_en[req_cols],df_hindi[req_cols]],ignore_index=True)

In [36]:
df_alpaca_hindi_mix_input_output_format = df_alpaca_hindi_mix_input_output_format.sample(frac=1,random_state=42).reset_index(drop=True)
df_alpaca_hindi_mix_input_output_format.to_csv(f'{output_location}df_alpaca_hindi_mix_input_output_format.csv',index=None)

In [38]:
# openhathi dataset
dataset = load_dataset("sarvamai/samvaad-hi-v1")

In [63]:
df = dataset['train'].to_pandas()
df.head()
# df.to_csv("sarvam_ai.csv",index=None)
# df = pd.read_csv('sarvam_ai.csv')

,messages
0,[{'content': 'Can you tell me about the 1988 R...
1,[{'content': 'Can you tell me more about the m...
2,[{'content': 'Mujhe Lavasa ke baare mein jaank...
3,[{'content': 'वैश्वीकरण भारत में स्वदेशी संस्क...
4,[{'content': 'Could you please tell me in Hind...


In [65]:
# df['text'] = df['messages'].apply(ast.literal_eval)
# df.head()

In [50]:
len(df['messages'].iloc[20])

6

In [66]:


def convert_json_to_gemma_instruct_format(json_data):
    text_format = ""
    for data in json_data:
        role = 'model' if data['role'] == 'assistant' else 'user'
        text_format += f"<start_of_turn>{role}\n{data['content']} <end_of_turn>model\n"
    return text_format

def convert_to_io_format(data):
    # Initialize a list to hold the input-output pairs
    io_pairs = []
    
    # Iterate through the provided data
    for i in range(0, len(data), 2):
        # Assuming the data alternates between user query and assistant answer,
        # add the pair to the io_pairs list.
        if data[i]['role'] == 'user' and i+1 < len(data) and data[i+1]['role'] == 'assistant':
            input_text = data[i]['content']
            output_text = data[i+1]['content']
            io_pairs.append((input_text, output_text))
    
    return io_pairs

def process_row(row, index):
    # Convert the row with the provided function
    io_pairs = convert_to_io_format(row.tolist())
    # Create a temporary DataFrame from the io_pairs
    temp_df = pd.DataFrame(io_pairs, columns=['input', 'output'])
    # Assign the index (which acts as the ID here)
    temp_df['id'] = index
    return temp_df
    

In [69]:
# Initialize an empty list to collect DataFrame pieces
data_pieces = []

# Use tqdm to show progress over the DataFrame's index and rows
for i in tqdm(df.index):
    row = df.loc[i]
    data_piece = process_row(row['messages'], i)
    data_pieces.append(data_piece)

# Concatenate all DataFrame pieces into one DataFrame
output_data = pd.concat(data_pieces, ignore_index=True)

100%|██████████| 101476/101476 [00:12<00:00, 8298.24it/s]


In [74]:
output_data= output_data.rename(columns={'id':'conversation_id'})
output_data['language']='hinglish'
output_data.head()
output_data.to_csv(f'{output_location}samvaad-hi-v1.csv',index=None)


In [101]:
# output_data

In [75]:
#gujarati all
dataset = load_dataset("jayshah5696/alpaca-gujarati-full")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 504.30it/s]
Generating train split: 51963 examples [00:01, 39844.01 examples/s]


In [81]:
df = dataset['train'].to_pandas()
df.shape

(51963, 6)

In [82]:
df.head()

,instruction,input,output,text,system_prompt,text_chat
0,મૂળ શબ્દોનો ઉપયોગ કરીને નીચેના વાક્યને ફરીથી લખો.,મારી તબીયત સારી લાગતી નથી.,મને સારું નથી લાગતું.,"નીચે એક સૂચના છે જે કાર્યનું વર્ણન કરે છે, ઇનપ...","નીચે એક સૂચના છે જે કાર્યનું વર્ણન કરે છે, ઇનપ...",<s>[INST]<<SYS>>\n નીચે એક સૂચના છે જે ...
1,બે વાક્યોને એક વાક્યમાં જોડો.,તે પાર્ટીમાં ગયો ન હતો.,"તે પાર્ટીમાં ગયો ન હતો, તેના બદલે ઘરે જ રહ્યો ...","નીચે એક સૂચના છે જે કાર્યનું વર્ણન કરે છે, ઇનપ...","નીચે એક સૂચના છે જે કાર્યનું વર્ણન કરે છે, ઇનપ...",<s>[INST]<<SYS>>\n નીચે એક સૂચના છે જે ...
2,નીચેના વાક્યને બે અલગ-અલગ વાક્યોમાં વિભાજિત કરો.,તેણે હેલો કહ્યું અને વિદાય લીધી.,તેણે હેલો કહ્યું.,"નીચે એક સૂચના છે જે કાર્યનું વર્ણન કરે છે, ઇનપ...","નીચે એક સૂચના છે જે કાર્યનું વર્ણન કરે છે, ઇનપ...",<s>[INST]<<SYS>>\n નીચે એક સૂચના છે જે ...
3,"નીચેના વાક્યને હકારાત્મક, નકારાત્મક અથવા તટસ્થ...",હું કાલે સ્ટોર પર જાઉં છું.,હકારાત્મક.,"નીચે એક સૂચના છે જે કાર્યનું વર્ણન કરે છે, ઇનપ...","નીચે એક સૂચના છે જે કાર્યનું વર્ણન કરે છે, ઇનપ...",<s>[INST]<<SYS>>\n નીચે એક સૂચના છે જે ...
4,વાક્યમાં કયો શબ્દ સૌથી વધુ વજન ધરાવે છે તે શોધો.,તેણે ડિનર માટે ચાઈનીઝ ફૂડનો ઓર્ડર આપ્યો.,રાત્રિભોજન.,"નીચે એક સૂચના છે જે કાર્યનું વર્ણન કરે છે, ઇનપ...","નીચે એક સૂચના છે જે કાર્યનું વર્ણન કરે છે, ઇનપ...",<s>[INST]<<SYS>>\n નીચે એક સૂચના છે જે ...


In [83]:
df['input'] = df.apply(lambda x: x['instruction'] if pd.isna(x['input']) else x['instruction']+'\n'+x['input'],axis=1)
df['language']='gujarati'
print(req_cols)
df = df[req_cols]

['input', 'output', 'language']


In [84]:
df.head()

,input,output,language
0,મૂળ શબ્દોનો ઉપયોગ કરીને નીચેના વાક્યને ફરીથી લ...,મને સારું નથી લાગતું.,gujarati
1,બે વાક્યોને એક વાક્યમાં જોડો.\nતે પાર્ટીમાં ગય...,"તે પાર્ટીમાં ગયો ન હતો, તેના બદલે ઘરે જ રહ્યો ...",gujarati
2,નીચેના વાક્યને બે અલગ-અલગ વાક્યોમાં વિભાજિત કર...,તેણે હેલો કહ્યું.,gujarati
3,"નીચેના વાક્યને હકારાત્મક, નકારાત્મક અથવા તટસ્થ...",હકારાત્મક.,gujarati
4,વાક્યમાં કયો શબ્દ સૌથી વધુ વજન ધરાવે છે તે શોધ...,રાત્રિભોજન.,gujarati


In [85]:
df.to_csv(f"{output_location}/alpaca_gujarati)all.csv",index=None)

In [86]:
dataset =load_dataset("CohereForAI/aya_dataset")

Generating test split: 100%|██████████| 1750/1750 [00:00<00:00, 618994.10 examples/s]


In [102]:
df = pd.concat([dataset['train'].to_pandas(),dataset['test'].to_pandas()])
df.head()

,inputs,targets,language,language_code,annotation_type,user_id
0,Heestan waxaa qada Khalid Haref Ahmed \nOO ku ...,Habeen ma hurdoo\nAday horjoogoo\nDharaar ma h...,Somali,som,original-annotations,f0ff69570af705b75c5a0851883e502feab2bc874c5e98...
1,Quels président des États-Unis ne s’est jamais...,James Buchanan est le seul président qui ne s'...,French,fra,re-annotations,5bd125e28d277d4fbc168f7accbcd05477951aea60008b...
2,كم عدد الخلفاء الراشدين ؟ أجب على السؤال السابق.,الإجابة هي: 4 خلفاء هم: أبوبكر الصديق، عمر بن ...,Standard Arabic,arb,original-annotations,13c05f2d3912479e80ce8aea399db395ce766b841c936c...
3,கேள்வி: \nகீழ்க்காணும் புதிருக்கான விடையை கண்...,பதில்: \nபடகு,Tamil,tam,re-annotations,91d8bea5f24cef25452aa5bca663c116e59d8265478b9a...
4,බුදු දහමේ එන රසය ගැන විස්තර කරන්න.,ජිහ්වා ප්‍ර‍සාදය නිසා එහි උපදනා විඥානයට දැනෙන්...,Sinhala,sin,original-annotations,29f22cf193a81e1a5c47d76af453a91b3cd19aa348995c...


In [103]:
df['language'].unique()

array(['Somali', 'French', 'Standard Arabic', 'Tamil', 'Sinhala',
       'Yoruba', 'Telugu', 'Sundanese', 'Italian', 'Spanish',
       'Portuguese', 'Vietnamese', 'Urdu', 'Marathi',
       'Modern Greek (1453-)', 'Dutch', 'Gujarati', 'Standard Malay',
       'Moroccan Arabic', 'Amharic', 'Turkish', 'Simplified Chinese',
       'Malayalam', 'Plateau Malagasy', 'Wolof', 'Hausa', 'Indonesian',
       'Panjabi', 'Kyrgyz', 'Nyanja', 'Nepali (individual language)',
       'Japanese', 'Ukrainian', 'Southern Pashto', 'English',
       'Najdi Arabic', 'Shona', 'Cebuano', 'German', 'Zulu', 'Polish',
       'Swahili (individual language)', 'Iranian Persian', 'Lithuanian',
       'Thai', 'Bengali', 'Filipino', 'Basque', 'Burmese', 'Igbo',
       'Danish', 'Russian', 'Hindi', 'Tosk Albanian', 'Kannada',
       'Traditional Chinese', 'Swedish', 'Haitian', 'Irish', 'Javanese',
       'Finnish', 'Korean', "Ta'izzi-Adeni Arabic", 'Sindhi',
       'Egyptian Arabic', 'Hungarian', 'Xhosa', 'Central Kurdis

In [104]:
selected_language=['Gujarati','Hindi']
df = df[df['language'].isin(selected_language)]

In [105]:
df = df.rename(columns={'inputs':"input",
                   'targets':"output",
                   'user_id':'conversation_id'})
df = df[req_cols+['conversation_id']]

In [107]:
df.to_csv(f"{output_location}aya_guj_hindi.csv",index=None)

In [108]:
dataset = load_dataset("el2e10/aya-indicsentiment-gujarati")
df = dataset['train'].to_pandas()
df.head()

Generating train split: 100%|██████████| 1156/1156 [00:00<00:00, 235095.78 examples/s]


,inputs,targets,template_lang,template_id
0,What's the Gujarati translation of this senten...,"""આ બોટનો સાઉન્ડબાર હજુ પણ તમામ સ્પીકર્સ માટે વ...",['eng'],3
1,What's the Gujarati translation of this senten...,"""માઇક અને માઇક્રો એસડી કાર્ડ સ્લોટ સાથે ફોલ્ડ ...",['eng'],3
2,"Translate this sentence to Gujarati: ""The rece...","""વાર્તાઓની તાજેતરમાં ઉમેરવામાં આવેલી વિશેષતા, ...",['eng'],2
3,"Translate from English to Gujarati: ""Rates are...","""દરો સ્પર્ધાત્મક હોય છે, લગભગ હંમેશા બજારમાં શ...",['eng'],1
4,What's the Gujarati translation of this senten...,"""ખૂબ જ મોટું અને કાર્યક્ષમ લાગે છે. પરંતુ આગળન...",['eng'],3


In [109]:
df = df.rename(columns={'inputs':'input','targets':'output'})

In [110]:
df['language']='gujarati'
df = df[req_cols]
df.to_csv(f"{output_location}aya_indicsentiment_gujarati.csv",index=None)

In [111]:
dataset = load_dataset("el2e10/aya-paraphrase-gujarati")
df = dataset['train'].to_pandas()
df.head()

Generating train split: 100%|██████████| 1001/1001 [00:00<00:00, 179969.06 examples/s]


,inputs,targets,template_lang,template_id
0,"નીચેના વાક્યને બીજા સ્વરૂપમાં ફરીથી લખો: ""ઘટના...","""પોલીસ ત્યારે ઘટના સ્થળે પહોંચી હતી જ્યારે ઘટન...",['guj'],3
1,"નીચેના વાક્યને અલગ રીતે ફરીથી લખો: ""આ ફિલ્મ મા...","""તેમને આ ફિલ્મ માટે ફિલ્મફેરનો બેસ્ટ સપોર્ટિંગ...",['guj'],2
2,"નીચેના વાક્યને અલગ શબ્દોનો ઉપયોગ કરીને લખો: ""ચ...","""ચંદીગઢ: પંજાબી સાહિત્યકાર જસવંત સિંગ કાનવાલનુ...",['guj'],1
3,"નીચેના વાક્યને અલગ શબ્દોનો ઉપયોગ કરીને લખો: ""શ...","""રાજ્યના મુખ્યમંત્રી ઉદ્ધવ ઠાકરે છે જે શિવ સેન...",['guj'],1
4,"નીચેના વાક્યને અલગ શબ્દોનો ઉપયોગ કરીને લખો: ""એ...","""આ સ્પર્ધામાં એસ.એસ.એલ.સી., પી.યુ., આઇ.ટી.આઇ.,...",['guj'],1


In [112]:
df = df.rename(columns={'inputs':'input','targets':'output'})
df['language']='gujarati'
df = df[req_cols]
df.to_csv(f"{output_location}aya_paraphrase_gujarati.csv",index=None)